<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [54]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [55]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [73]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [74]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [75]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_0

,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0),y1(0),y2(0),y3(0)
0,0.473645,0.275821,0.989537,5.051251,-68.099912,1.140492,0.122876,0.953330,1.140492
1,0.422421,-0.997436,0.945394,5.756407,-10.410296,1.358041,0.047756,0.988662,1.358041
2,0.107987,0.087687,-0.381604,-0.419400,-5.853073,3.548152,-0.285736,0.628849,3.548152
3,-0.142399,-0.262149,-0.672332,-7.283922,84.363663,1.154650,0.014409,0.956375,1.154650
4,0.049123,-0.201346,-0.857060,246.803540,-71.739176,2.563907,0.000098,0.738977,2.563907
...,...,...,...,...,...,...,...,...,...
95,-0.984082,0.523287,0.979129,-3.104024,-14.082891,0.549658,0.434763,0.722769,0.549658
96,-0.108210,-0.924593,-0.885956,-5.521451,-70.660717,8.601615,0.006705,0.856330,8.601615
97,-0.304222,-0.839631,-0.593749,-3.318568,80.638944,-0.082132,0.055181,0.286426,-0.082132
98,0.051224,-0.800767,-0.303003,5.630466,27.301788,1.256355,0.002301,0.975177,1.256355


In [59]:
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_1

,x1(1),x2(1),x3(1),v1(1),v2(1),v3(1),y1(1),y2(1),y3(1)
0,-0.967453,-0.995061,-0.616455,20.453380,23.756840,3.796422,-0.046640,0.780400,3.796422
1,-0.780126,-0.971092,-0.602494,7.187527,-52.813112,9.316434,-0.097283,0.328835,9.316434
2,0.168277,0.862468,0.616609,1.592602,-55.392502,1.597963,0.046672,0.999815,1.597963
3,-0.608751,-0.996937,-0.989337,0.116063,44.080833,4.682230,-4.098566,0.999773,4.682230
4,-0.613731,0.982487,-0.325112,1.823432,-65.905814,0.715125,-0.266020,0.809760,0.715125
...,...,...,...,...,...,...,...,...,...
95,-0.919170,-0.740952,-0.959790,-10.470723,-22.720734,-2.959736,0.097774,0.425272,-2.959736
96,-0.174026,0.156977,-0.118247,26.585408,-101.372643,1.010607,-0.006892,0.920410,1.010607
97,0.324464,-0.782004,-0.412923,-6.437948,-92.484985,-0.595953,-0.032464,0.749198,-0.595953
98,-0.999659,0.487046,0.824197,0.117545,-46.662558,1.562413,-12.183998,0.999982,1.562413


In [60]:
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_2

,x1(2),x2(2),x3(2),v1(2),v2(2),v3(2),y1(2),y2(2),y3(2)
0,0.136394,-0.447895,-0.982721,4.536822,-27.337263,0.719318,0.016590,0.811709,0.719318
1,-0.057874,-0.615998,0.440984,9.221781,-28.764919,2.289375,-0.001924,0.867607,2.289375
2,0.179046,-0.182678,-0.410322,2.885681,-52.532879,1.312979,0.061426,0.983335,1.312979
3,0.634914,0.419366,-0.307408,7.850196,72.313768,1.047134,0.099516,0.930588,1.047134
4,-0.227722,-0.297275,-0.980265,180.395499,46.135805,15.239424,-0.001105,0.672000,15.239424
...,...,...,...,...,...,...,...,...,...
95,0.485929,-0.986446,0.069794,3.077393,-96.689585,11.953392,0.110825,0.758490,11.953392
96,0.935857,0.693783,0.997894,0.142732,59.557432,-0.062851,7.615175,0.250618,-0.062851
97,0.991455,-0.546685,-0.974684,1.216296,-21.882940,18.948687,1.097746,0.314593,18.948687
98,-0.998607,0.994014,-0.677139,403.590505,-42.350878,2.953821,-0.002480,0.432053,2.953821


##1. Линейная форма функции пересчёта

In [76]:
vector_ones = [1] * elem

###Для функции $x_1[t+1]$

In [79]:
#x3, v1, v2
x1_linearFunc = model_0[['x3(0)', 'v1(0)', 'v2(0)']]
x1_linearFunc['x1(1)'] = model_1['x1(1)']
x1_linearFunc.insert(0, "optional_col", vector_ones)

A1_linear = np.array(x1_linearFunc)
#A1_psev = BenIsrael(A1_linear[:, 0:(len(A1_linear[0]) - 1)])
X1_linear = solvingSLAE(A1_linear[:, 0:(len(A1_linear[0]) - 1)], np.matrix(A1_linear[:, len(A1_linear[0]) - 1]).T)
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c13', 'b11', 'b12']))

           X1
a1  -0.053479
c13 -0.048331
b11  0.000093
b12  0.000090


<ipython-input-79-9c8f13bf7d14>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['x1(1)'] = model_1['x1(1)']


In [62]:
X1_estimate_linear = []
X1_variance_linear = []
X1_error_linear = []
for i in range(len(x1_linearFunc)):
  X1_estimate_linear.append(X1_linear[0,0] + X1_linear[1,0] * np.array(x1_linearFunc)[i,1]
                            + X1_linear[2,0] * np.array(x1_linearFunc)[i,2] + X1_linear[3,0] * np.array(x1_linearFunc)[i,3])

  X1_variance_linear.append((1.0 / len(x1_linearFunc)) * (np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1] - X1_estimate_linear[i])**2)

  X1_error_linear.append((1.0 / len(x1_linearFunc)) * abs((np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1] - X1_estimate_linear[i])
                            / np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1]) * 100)

x1_linearFunc['X1*'] = X1_estimate_linear
x1_linearFunc['(X1 - X1*)^2'] = X1_variance_linear
x1_linearFunc['|(X1 - X1*)| / |X1|, %'] = X1_error_linear
x1_linearFunc

<ipython-input-62-65ca3a007bdb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['X1*'] = X1_estimate_linear
<ipython-input-62-65ca3a007bdb>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['(X1 - X1*)^2'] = X1_variance_linear
<ipython-input-62-65ca3a007bdb>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,x3(0),v1(0),v2(0),x1(1),X1*,(X1 - X1*)^2,"|(X1 - X1*)| / |X1|, %"
0,1,-0.608865,-2.747214,5.788165,-0.967453,0.198551,0.013596,1.205231
1,1,0.159990,-6.457003,20.700391,-0.780126,-0.183517,0.003559,0.764760
2,1,-0.903267,0.661840,-20.555920,0.168277,0.363095,0.000380,1.157724
3,1,0.354955,0.834953,-36.625400,-0.608751,-0.225802,0.001466,0.629073
4,1,-0.920850,6.782785,3.037581,-0.613731,0.352668,0.009339,1.574629
...,...,...,...,...,...,...,...,...
95,1,0.486092,4.839251,7.191159,-0.919170,-0.325400,0.003526,0.645985
96,1,-0.683074,2.678620,79.702885,-0.174026,0.171749,0.001196,1.986913
97,1,0.218791,2.501724,-68.949353,0.324464,-0.132333,0.002087,1.407851
98,1,0.528252,2.006046,9.882040,-0.999659,-0.348594,0.004239,0.651287


###Для функции $x_2[t+1]$

In [63]:
#x2, x3, v1
x2_linearFunc = model_0[['x2(0)', 'x3(0)', 'v1(0)']]
x2_linearFunc['x2(1)'] = model_1['x2(1)']
x2_linearFunc.insert(0, "optional_col", vector_ones)

A2_linear = np.array(x2_linearFunc)
#A2_psev = BenIsrael(A2_linear[:, 0:(len(A2_linear[0]) - 1)])
X2_linear = solvingSLAE(A2_linear[:, 0:(len(A2_linear[0]) - 1)], np.matrix(A2_linear[:, len(A2_linear[0]) - 1]).T)
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))

           X2
a2  -0.012878
c22 -0.032119
c23 -0.141523
b21  0.000020


<ipython-input-63-39802f8807a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linearFunc['x2(1)'] = model_1['x2(1)']


In [64]:
X2_estimate_linear = []
X2_variance_linear = []
X2_error_linear = []
for i in range(len(x2_linearFunc)):
  X2_estimate_linear.append(X2_linear[0,0] + X2_linear[1,0] * np.array(x2_linearFunc)[i,1]
                            + X2_linear[2,0] * np.array(x2_linearFunc)[i,2] + X2_linear[3,0] * np.array(x2_linearFunc)[i,3])

  X2_variance_linear.append((1.0 / len(x2_linearFunc)) * (np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1] - X2_estimate_linear[i])**2)

  X2_error_linear.append((1.0 / len(x2_linearFunc)) * abs((np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1] - X2_estimate_linear[i])
                            / np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1]) * 100)

x2_linearFunc['X2*'] = X2_estimate_linear
x2_linearFunc['(X2 - X2*)^2'] = X2_variance_linear
x2_linearFunc['|(X2 - X2*)| / |X2|, %'] = X2_error_linear
x2_linearFunc

<ipython-input-64-75044911546d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linearFunc['X2*'] = X2_estimate_linear
<ipython-input-64-75044911546d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linearFunc['(X2 - X2*)^2'] = X2_variance_linear
<ipython-input-64-75044911546d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,x2(0),x3(0),v1(0),x2(1),X2*,(X2 - X2*)^2,"|(X2 - X2*)| / |X2|, %"
0,1,0.707405,-0.608865,-2.747214,-0.995061,0.050514,0.010932,1.050765
1,1,0.888914,0.159990,-6.457003,-0.971092,-0.064202,0.008224,0.933887
2,1,0.367135,-0.903267,0.661840,0.862468,0.103176,0.005765,0.880371
3,1,0.755512,0.354955,0.834953,-0.996937,-0.087362,0.008273,0.912370
4,1,0.489459,-0.920850,6.782785,0.982487,0.101859,0.007755,0.896325
...,...,...,...,...,...,...,...,...
95,1,-0.182549,0.486092,4.839251,-0.740952,-0.075710,0.004425,0.897821
96,1,0.510132,-0.683074,2.678620,0.156977,0.067462,0.000080,0.570245
97,1,-0.549400,0.218791,2.501724,-0.782004,-0.026145,0.005713,0.966567
98,1,-0.070682,0.528252,2.006046,0.487046,-0.085327,0.003276,1.175192


###Для функции $x_3[t+1]$

In [65]:
#x3, v1, v3
x3_linearFunc = model_0[['x3(0)', 'v1(0)', 'v3(0)']]
x3_linearFunc['x3(1)'] = model_1['x3(1)']
x3_linearFunc.insert(0, "optional_col", vector_ones)

A3_linear = np.array(x3_linearFunc)
#A3_psev = BenIsrael(A3_linear[:, 0:(len(A3_linear[0]) - 1)])
X3_linear = solvingSLAE(A3_linear[:, 0:(len(A3_linear[0]) - 1)], np.matrix(A3_linear[:, len(A3_linear[0]) - 1]).T)
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))

           X3
a3  -0.169104
c33  0.028765
b31  0.000684
b33 -0.011492


<ipython-input-65-f740405435c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['x3(1)'] = model_1['x3(1)']


In [66]:
X3_estimate_linear = []
X3_variance_linear = []
X3_error_linear = []
for i in range(len(x3_linearFunc)):
  X3_estimate_linear.append(X3_linear[0,0] + X3_linear[1,0] * np.array(x3_linearFunc)[i,1]
                            + X3_linear[2,0] * np.array(x3_linearFunc)[i,2] + X3_linear[3,0] * np.array(x3_linearFunc)[i,3])

  X3_variance_linear.append((1.0 / len(x3_linearFunc)) * (np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1] - X3_estimate_linear[i])**2)

  X3_error_linear.append((1.0 / len(x3_linearFunc)) * abs((np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1] - X3_estimate_linear[i])
                            / np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1]) * 100)

x3_linearFunc['X3*'] = X3_estimate_linear
x3_linearFunc['(X3 - X3*)^2'] = X3_variance_linear
x3_linearFunc['|(X3 - X3*)| / |X3|, %'] = X3_error_linear
x3_linearFunc

<ipython-input-66-0f73eb653e66>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['X3*'] = X3_estimate_linear
<ipython-input-66-0f73eb653e66>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['(X3 - X3*)^2'] = X3_variance_linear
<ipython-input-66-0f73eb653e66>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,x3(0),v1(0),v3(0),x3(1),X3*,(X3 - X3*)^2,"|(X3 - X3*)| / |X3|, %"
0,1,-0.608865,-2.747214,18.216359,-0.616455,-0.397842,0.000478,0.354629
1,1,0.159990,-6.457003,0.262125,-0.602494,-0.171933,0.001854,0.714631
2,1,-0.903267,0.661840,0.536725,0.616609,-0.200802,0.006682,1.325655
3,1,0.354955,0.834953,15.982961,-0.989337,-0.342000,0.004190,0.654314
4,1,-0.920850,6.782785,2.136938,-0.325112,-0.215509,0.000120,0.337125
...,...,...,...,...,...,...,...,...
95,1,0.486092,4.839251,12.182477,-0.959790,-0.291812,0.004462,0.695963
96,1,-0.683074,2.678620,7.880341,-0.118247,-0.277481,0.000254,1.346624
97,1,0.218791,2.501724,1.149470,-0.412923,-0.174308,0.000569,0.577867
98,1,0.528252,2.006046,0.391899,0.824197,-0.157040,0.009628,1.190537


###Для функции $y_1[t]$

In [67]:
#x1, x2, v1
y1_linearFunc = model_0[['x1(0)', 'x2(0)', 'v1(0)']]
y1_linearFunc['y1(0)'] = model_0['y1(0)']
y1_linearFunc.insert(0, "optional_col", vector_ones)

A1y_linear = np.array(y1_linearFunc)
#A1y_psev = BenIsrael(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)])
Y1_linear = solvingSLAE(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)], np.matrix(A1y_linear[:, len(A1y_linear[0]) - 1]).T)
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))

           Y1
a1  -0.107822
c11  0.212420
c12  0.044895
b11  0.000174


<ipython-input-67-2e41166f56a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['y1(0)'] = model_0['y1(0)']


In [68]:
Y1_estimate_linear = []
Y1_variance_linear = []
Y1_error_linear = []
for i in range(len(y1_linearFunc)):
  Y1_estimate_linear.append(Y1_linear[0,0] + Y1_linear[1,0] * np.array(y1_linearFunc)[i,1]
                            + Y1_linear[2,0] * np.array(y1_linearFunc)[i,2] + Y1_linear[3,0] * np.array(y1_linearFunc)[i,3])

  Y1_variance_linear.append((1.0 / len(y1_linearFunc)) * (np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1] - Y1_estimate_linear[i])**2)

  Y1_error_linear.append((1.0 / len(y1_linearFunc)) * abs((np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1] - Y1_estimate_linear[i])
                            / np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1]) * 100)

y1_linearFunc['Y1*'] = Y1_estimate_linear
y1_linearFunc['(Y1 - Y1*)^2'] = Y1_variance_linear
y1_linearFunc['|(Y1 - Y1*)| / |Y1|, %'] = Y1_error_linear
y1_linearFunc

<ipython-input-68-20857d005a00>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['Y1*'] = Y1_estimate_linear
<ipython-input-68-20857d005a00>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['(Y1 - Y1*)^2'] = Y1_variance_linear
<ipython-input-68-20857d005a00>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,x1(0),x2(0),v1(0),y1(0),Y1*,(Y1 - Y1*)^2,"|(Y1 - Y1*)| / |Y1|, %"
0,1,0.841227,0.707405,-2.747214,-0.333920,0.102152,0.001902,1.305918
1,1,0.626052,0.888914,-6.457003,-0.081368,0.063949,0.000211,1.785919
2,1,-0.700762,0.367135,0.661840,-1.462817,-0.240080,0.014951,0.835878
3,1,0.285583,0.755512,0.834953,0.210289,-0.013094,0.000499,1.062269
4,1,-0.159008,0.489459,6.782785,-0.013362,-0.118444,0.000110,7.864453
...,...,...,...,...,...,...,...,...
95,1,0.017287,-0.182549,4.839251,0.001099,-0.111504,0.000127,102.433561
96,1,-0.875200,0.510132,2.678620,-0.427965,-0.270363,0.000248,0.368259
97,1,0.657977,-0.549400,2.501724,0.287827,0.007715,0.000785,0.973196
98,1,-0.620962,-0.070682,2.006046,-0.917494,-0.242551,0.004555,0.735637


###Для функции $y_2[t]$

In [69]:
#v3
y2_linearFunc = model_0[['v3(0)']]
y2_linearFunc['y2(0)'] = model_0['y2(0)']
y2_linearFunc.insert(0, "optional_col", vector_ones)

A2y_linear = np.array(y2_linearFunc)
#A2y_psev = BenIsrael(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)])
Y2_linear = solvingSLAE(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)], np.matrix(A2y_linear[:, len(A2y_linear[0]) - 1]).T)
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b23']))

           Y2
a2   0.738052
b23  0.005251


<ipython-input-69-a0e07d78ce80>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['y2(0)'] = model_0['y2(0)']


In [70]:
Y2_estimate_linear = []
Y2_variance_linear = []
Y2_error_linear = []
for i in range(len(y2_linearFunc)):
  Y2_estimate_linear.append(Y2_linear[0,0] + Y2_linear[1,0] * np.array(y2_linearFunc)[i,1])

  Y2_variance_linear.append((1.0 / len(y2_linearFunc)) * (np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1] - Y2_estimate_linear[i])**2)

  Y2_error_linear.append((1.0 / len(y2_linearFunc)) * abs((np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1] - Y2_estimate_linear[i])
                            / np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1]) * 100)

y2_linearFunc['Y2*'] = Y2_estimate_linear
y2_linearFunc['(Y2 - Y2*)^2'] = Y2_variance_linear
y2_linearFunc['|(Y2 - Y2*)| / |Y2|, %'] = Y2_error_linear
y2_linearFunc

<ipython-input-70-7cd0bf4dc686>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['Y2*'] = Y2_estimate_linear
<ipython-input-70-7cd0bf4dc686>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['(Y2 - Y2*)^2'] = Y2_variance_linear
<ipython-input-70-7cd0bf4dc686>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,v3(0),y2(0),Y2*,(Y2 - Y2*)^2,"|(Y2 - Y2*)| / |Y2|, %"
0,1,18.216359,0.769237,0.833702,0.000042,0.083805
1,1,0.262125,0.509051,0.739428,0.000531,0.452561
2,1,0.536725,0.715070,0.740870,0.000007,0.036081
3,1,15.982961,0.521099,0.821975,0.000905,0.577386
4,1,2.136938,0.918682,0.749272,0.000287,0.184405
...,...,...,...,...,...,...
95,1,12.182477,0.611992,0.802019,0.000361,0.310507
96,1,7.880341,0.999826,0.779430,0.000486,0.220435
97,1,1.149470,0.955273,0.744087,0.000446,0.221074
98,1,0.391899,0.618017,0.740109,0.000149,0.197556


###Для функции $y_3[t]$

In [71]:
#v1, v3
y3_linearFunc = model_0[['v1(0)', 'v3(0)']]
y3_linearFunc['y3(0)'] = model_0['y3(0)']
y3_linearFunc.insert(0, "optional_col", vector_ones)

A3y_linear = np.array(y3_linearFunc)
#A3y_psev = BenIsrael(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)])
Y3_linear = solvingSLAE(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)], np.matrix(A3y_linear[:, len(A3y_linear[0]) - 1]).T)
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))

               Y3
a3   2.728482e-08
b31 -2.415186e-11
b33  1.000000e+00


<ipython-input-71-b0b21a59e704>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['y3(0)'] = model_0['y3(0)']


In [72]:
Y3_estimate_linear = []
Y3_variance_linear = []
Y3_error_linear = []
for i in range(len(y3_linearFunc)):
  Y3_estimate_linear.append(Y3_linear[0,0] + Y3_linear[1,0] * np.array(y3_linearFunc)[i,1]
                            + Y3_linear[2,0] * np.array(y3_linearFunc)[i,2])

  Y3_variance_linear.append((1.0 / len(y3_linearFunc)) * (np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1] - Y3_estimate_linear[i])**2)

  Y3_error_linear.append((1.0 / len(y3_linearFunc)) * abs((np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1] - Y3_estimate_linear[i])
                            / np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1]) * 100)

y3_linearFunc['Y3*'] = Y3_estimate_linear
y3_linearFunc['(Y3 - Y3*)^2'] = Y3_variance_linear
y3_linearFunc['|(Y3 - Y3*)| / |Y3|, %'] = Y3_error_linear
y3_linearFunc

<ipython-input-72-e73931a3ed05>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['Y3*'] = Y3_estimate_linear
<ipython-input-72-e73931a3ed05>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['(Y3 - Y3*)^2'] = Y3_variance_linear
<ipython-input-72-e73931a3ed05>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,optional_col,v1(0),v3(0),y3(0),Y3*,(Y3 - Y3*)^2,"|(Y3 - Y3*)| / |Y3|, %"
0,1,-2.747214,18.216359,18.216359,18.216359,2.032901e-18,7.827022e-10
1,1,-6.457003,0.262125,0.262125,0.262125,7.204946e-18,1.024018e-07
2,1,0.661840,0.536725,0.536725,0.536725,6.782308e-18,4.852178e-08
3,1,0.834953,15.982961,15.982961,15.982961,8.543405e-19,5.783068e-10
4,1,6.782785,2.136938,2.136938,2.136938,4.946124e-18,1.040736e-08
...,...,...,...,...,...,...,...
95,1,4.839251,12.182477,12.182477,12.182477,4.340632e-21,5.408053e-11
96,1,2.678620,7.880341,7.880341,7.880341,8.501084e-19,1.170017e-09
97,1,2.501724,1.149470,1.149470,1.149470,6.051018e-18,2.140013e-08
98,1,2.006046,0.391899,0.391899,0.391899,6.938591e-18,6.721421e-08
